### Binary classification based on 3 layers neural network
#### author: Kim Jeong Min

##### load images

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pl

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import argparse
import sys
import os
import numpy as np
import time
import datetime 
import csv
import configparser
import argparse
import platform

from torch.autograd import Variable
from random import shuffle

import math

In [ ]:
class Linear(nn.Module):

    def __init__(self, num_classes=2):

        super(Linear, self).__init__()

        self.number_class   = num_classes

        _size_image     = 100* 100
        _num1           = 400
        _num2           = 150
        
        self.fc1        = nn.Linear(_size_image, _num1, bias=True)
        self.fc2        = nn.Linear(_num1, _num2, bias=True)
        self.fc3        = nn.Linear(_num2, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.PReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.PReLU(True))
        self.fc_layer3  = nn.Sequential(self.fc3, nn.Sigmoid(True))
        
        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2, self.fc_layer3)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for m in self.modules():
            
            n = m.in_features
            m.weight.data.uniform_(- 1.0 / math.sqrt(n), 1.0 / math.sqrt(n))

            if m.bias is not None:

                m.bias.data.zero_()

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


In [2]:
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = 'horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)  


validation_data_path = 'horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=1)  

IMAGE_VECTOR_SIZE = 10000

train_data = np.empty((IMAGE_VECTOR_SIZE,0))
train_label = np.empty((0,1))
validation_data = np.empty((IMAGE_VECTOR_SIZE,0))
validation_label = np.empty((0,1))

# load training images of the batch size for every iteration
for i, data in enumerate(trainloader):
    # inputs is the image
    # labels is the class of the image
    inputs, labels = data
    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    train_data = np.hstack((train_data, np.reshape(inputs, (10000,1))))
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    # human: 1, horse: 0
    train_label = np.append(train_label, 1 if sum(labels)>0 else 0)  

# load validation images of the batch size for every iteration
for i, data in enumerate(valloader):
    # inputs is the image
    # labels is the class of the image
    inputs, labels = data
    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
    validation_data = np.hstack((validation_data, np.reshape(inputs, (10000,1))))
    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    # human: 1, horse: 0
    validation_label = np.append(validation_label, 1 if sum(labels)>0 else 0)

num_classes = 2

##### load neural network model

In [4]:
model = Linear(num_classes=num_classes)

ModuleNotFoundError: No module named 'Linear'

##### Set the flag for using cuda

In [ ]:
bCuda = 1
if bCuda:
    model.cuda()

##### optimization algorithm

In [ ]:
optimizer   = optim.SGD(model.parameters())
objective   = nn.CrossEntropyLoss()

##### function for training the model

In [ ]:
def train():
    # print('train the model at given epoch')
    loss_train          = []
    model.train()
    for idx_batch, (data, target) in enumerate(loader_train):
        if bCuda:
            data, target    = data.cuda(), target.cuda()
            
        data, target    = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        output  = model(data)
        loss    = objective(output, target)

        loss.backward()
        optimizer.step()

        loss_train_batch    = loss.item() / len(data)
        loss_train.append(loss_train_batch)
        
    loss_train_mean     = np.mean(loss_train)
    loss_train_std      = np.std(loss_train)

    return {'loss_train_mean': loss_train_mean, 'loss_train_std': loss_train_std}

##### function for testing the model

In [ ]:
def test():
    # print('test the model at given epoch')

    accuracy_test   = []
    loss_test       = 0
    correct         = 0

    model.eval()

    for idx_batch, (data, target) in enumerate(loader_test):

        if bCuda:
        
            data, target    = data.cuda(), target.cuda()

        data, target    = Variable(data), Variable(target)

        output  = model(data)
        loss    = objective(output, target)

        loss_test   += loss.item()
        pred        = output.data.max(1)[1]
        correct     += pred.eq(target.data.view_as(pred)).cpu().sum()

    loss_test       = loss_test / len(loader_test.dataset)
    accuracy_test   = 100. * float(correct) / len(loader_test.dataset)

    return {'loss_test': loss_test, 'accuracy_test': accuracy_test}

##### iteration for the epoch

In [ ]:
for e in range(epoch):
        
    result_train    = train()
    result_test     = test()

    loss_train_mean[e]  = result_train['loss_train_mean']
    loss_train_std[e]   = result_train['loss_train_std']
    loss_test[e]        = result_test['loss_test']
    accuracy_test[e]    = result_test['accuracy_test']